In [ ]:
%%capture
!pip install langchain==0.1.1 openai==1.8.0 langchain-openai duckduckgo-search wikipedia langchainhub faiss-cpu

In [ ]:
import os
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# 🤖 Agents in LangChain

**Agents** are the driving force in LangChain, smart systems that use language models to work with other digital tools, perfect for tasks like Q&A, API interactions, and more.

### 🆚 Agents vs. Chains

- **Agents** dynamically decide on actions using language models.
- **Chains** have a predetermined sequence of actions.

Agents are about smart decision-making and interacting with the digital environment, while chains are about a pre-set flow of information.

### 🛠️ Tools and Toolkits

- **Tools** are like an agent's skills, specific to tasks like data retrieval.
- **Toolkits** bundle these skills for more complex tasks.

### 🧠 The Role of Memory

- Memory gives agents the ability to remember and use past interactions, making them smarter over time.



In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI

# instantiate the llm
llm = ChatOpenAI(model_name="gpt-4-0125-preview",temperature=0.0)

# Giving the Agent Tools


 - 🛠️ 🔧 Tools are integrated into agents either all at once during setup  or individually as needed .

 - 🧰 Opt for LangChain's ready-made tools or craft your own custom tools to meet your unique needs.

You can find the available native tools [here](https://github.com/langchain-ai/langchain/blob/ccb9e3ee2d4ffde1bb33c6c0df0db87aff3341bf/libs/langchain/langchain/agents/load_tools.py#L409) and look at the dictionary `_EXTRA_OPTIONAL_TOOLS` for the key of the tool.

In [ ]:
# equip it with some tools
tools = load_tools(["ddg-search", "llm-math", "wikipedia"], llm=llm)

# Initialize the agent

**Setting Up an Agent:**

There are two main methods to bring your LangChain agent to life:

1. **AgentExecutor** 🛠️: Acts as the agent's "command center." It's where the agent's actions are executed and where its memory is managed. You'll need an agent, a toolkit, and an optional memory component to get started.

2. **Initialize Agent** 🚀: This is the streamlined method for a quick launch. Provide the agent class, language model, and any tools you want to use, and it sets everything up for you.



In [ ]:
query = """
Who is the current Chief AI Scientist at Meta AI? When was he born? Where was he born? What's the current temperature there?
"""

In [ ]:
# initialize the agent
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

agent.run(query)

### AgentExecutor

The `AgentExecutor` class is responsible for executing the agent's actions and managing the agent's memory.

It takes an agent, a set of tools, and an optional memory object as input.

In [ ]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model_name="gpt-4-0125-preview",temperature=0.0)

tools = load_tools(["ddg-search", "llm-math", "wikipedia"], llm=llm)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True)

In [ ]:
query = "What's the current weather in Winnipeg?"

agent_executor.invoke({"input":query})

# 🔄 **Agents and Memory**

### **Stateless Agent Characteristics**

- **Current Limitation**: Inability to recall past interactions by default.

### **Enabling Memory**:

- **Method**: Integrate previous chat history into the agent.

- **Key Requirement**: The chat history variable must be named "chat_history" for compatibility with the current prompt.

- **Flexibility**: Altering the prompt
allows for different variable naming.

🔑 **Takeaway**: Adjusting the prompt or incorporating chat history enables stateful interactions, enhancing the agent's capability to remember and utilize past interactions.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": query},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "What do you reckon I should wear then?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

# Creating agents in LangChain v0.2 and beyond

Starting in v0.2 LangChain will stop using `initialize_agent` in favor of individual agent methods, as these are more clear.

You can see the affected agents in the [changelog](https://python.langchain.com/docs/changelog/langchain).

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-0125-preview",temperature=0.0)

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vector.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

tools = [search, retriever_tool]

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})